### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lower_Grade_Glioma/GSE24072'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

is_gene_available = True

# Data availability
trait_row = 2 if len(set(['type: glioma grade III', 'type: glioma grade IV', 'type: glioma grade V'])) > 1 else None
age_row = 1 if len(set(['age: 72', 'age: 70', 'age: 34', 'age: 54', 'age: 68', 'age: 30', 'age: 60', 'age: 73', 'age: 52', 'age: 65', 'age: 76', 'age: 51', 'age: 43', 'age: 67', 'age: 66', 'age: 69', 'age: 74', 'age: 36', 'age: 38', 'age: 63', 'age: 46', 'age: 55'])) > 1 else None
gender_row = 0 if len(set(['gender: male', 'gender: female'])) > 1 else None

# Data type conversion functions
def convert_trait(value):
    if value:
        val = value.split(":")[1].strip()
        if val in ['glioma grade III', 'glioma grade IV', 'glioma grade V']:
            return val
    return None

def convert_age(value):
    try:
        val = int(value.split(":")[1].strip())
        return val
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    if value:
        val = value.split(":")[1].strip().lower()
        if val == 'female':
            return 0
        elif val == 'male':
            return 1
    return None

# Save cohort information
save_cohort_info('GSE24072', './preprocessed/Lower_Grade_Glioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lower_Grade_Glioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lower_Grade_Glioma/trait_data/GSE24072.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine the necessary keys for mapping.
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Lower_Grade_Glioma/gene_data/GSE24072.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# Debugging: Ensure merged data is correct.
print("Merged Data Summary:")
print(merged_data.head())   # Check the initial rows 
print("Data Types:\n", merged_data.dtypes)

# Data conversion to ensure correct formats for bias checking
def convert_data(df, column, conversion_func):
    if column in df.columns:
        df[column] = df[column].apply(conversion_func)
    return df

merged_data = convert_data(merged_data, 'Lower_Grade_Glioma', convert_trait)
merged_data = convert_data(merged_data, 'Age', convert_age)
merged_data = convert_data(merged_data, 'Gender', convert_gender)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Lower_Grade_Glioma')

# 4. Save the cohort information with appropriate values.
save_cohort_info('GSE24072', './preprocessed/Lower_Grade_Glioma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Lower_Grade_Glioma/GSE24072.csv'
    unbiased_merged_data.to_csv(csv_path)
